# Advanced AlphaEarth Analysis

This notebook covers advanced topics including:

- Custom band combinations and what they reveal
- Multiple location analysis
- Batch processing considerations
- Integration with other GIS workflows

---

**Attribution:** The AlphaEarth Foundations Satellite Embedding dataset is produced by Google and Google DeepMind.

## Setup

In [ ]:
import ee
import leafmap.maplibregl as leafmap

In [ ]:
# Initialize Earth Engine
# ee.Authenticate()
# ee.Initialize(project="YOUR_PROJECT_ID")

In [ ]:
# Load dataset
dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

## Custom Band Combinations

AlphaEarth has 64 embedding bands (A01-A64). Different band combinations can highlight different features.

### Understanding the Bands

The 64 bands are learned representations that capture various semantic features:
- Some bands respond strongly to vegetation
- Others highlight water bodies
- Some capture urban/built environments
- Others encode elevation or terrain features

Since these are learned embeddings (not traditional spectral bands), experimentation is key!

In [ ]:
def format_band_names(band_numbers):
    """Convert band numbers (1-64) to band names (A01-A64)."""
    return [f"A{str(n).zfill(2)}" for n in band_numbers]

def explore_bands(image, center, zoom, band_combo, name="Custom Bands"):
    """
    Create a map with custom band visualization.
    
    Args:
        image: ee.Image with AlphaEarth embeddings
        center: [lat, lon] center coordinates
        zoom: zoom level
        band_combo: list of 3 band numbers (1-64) for RGB
        name: layer name
    """
    m = leafmap.Map(center=center, zoom=zoom)
    m.add_basemap("USGS.Imagery")
    
    bands = format_band_names(band_combo)
    vis = {
        "bands": bands,
        "min": -0.3,
        "max": 0.3,
    }
    
    m.add_ee_layer(image, vis, name)
    m.add_layer_control()
    
    return m

In [ ]:
# Load 2024 image
image_2024 = dataset.filter(ee.Filter.eq("year", 2024)).first()

# Location
center = [37.8, -122.4]  # San Francisco
zoom = 12

### Band Combination Experiments

Try different combinations to see how they highlight different features:

In [ ]:
# Default combination (recommended)
m = explore_bands(image_2024, center, zoom, [1, 16, 9], "Bands 1, 16, 9")
m

In [ ]:
# Try different bands
m = explore_bands(image_2024, center, zoom, [5, 25, 45], "Bands 5, 25, 45")
m

In [ ]:
# Another combination
m = explore_bands(image_2024, center, zoom, [10, 30, 50], "Bands 10, 30, 50")
m

## Multiple Location Analysis

Analyze several locations of interest in sequence.

In [ ]:
# Define locations of interest
locations = {
    "San Francisco": {"lat": 37.8, "lon": -122.4},
    "New York City": {"lat": 40.7, "lon": -74.0},
    "Amazon Rainforest": {"lat": -3.0, "lon": -60.0},
    "Sahara Desert": {"lat": 25.0, "lon": 10.0},
    "Tokyo": {"lat": 35.7, "lon": 139.7},
}

In [ ]:
def analyze_location(name, coords, year1=2017, year2=2024):
    """
    Create change detection map for a location.
    """
    print(f"\n=== {name} ===")
    print(f"Location: {coords['lat']}, {coords['lon']}")
    print(f"Years: {year1} vs {year2}")
    
    # Load images
    img1 = dataset.filter(ee.Filter.eq("year", year1)).first()
    img2 = dataset.filter(ee.Filter.eq("year", year2)).first()
    
    # Create map
    m = leafmap.Map(center=[coords["lat"], coords["lon"]], zoom=10)
    m.add_basemap("USGS.Imagery")
    
    # Add embedding layers
    vis_params = {"bands": ["A01", "A16", "A09"], "min": -0.3, "max": 0.3}
    m.add_ee_layer(img1, vis_params, f"{year1}")
    m.add_ee_layer(img2, vis_params, f"{year2}")
    
    # Calculate and add change detection
    band_names = [f"A{str(i).zfill(2)}" for i in range(1, 65)]
    similarity = img1.select(band_names).multiply(img2.select(band_names)).reduce(ee.Reducer.sum()).clamp(0, 1)
    m.add_ee_layer(similarity, {"min": 0, "max": 1, "palette": ["white", "black"]}, "Change")
    
    m.add_layer_control()
    return m

In [ ]:
# Analyze San Francisco
m = analyze_location("San Francisco", locations["San Francisco"])
m

In [ ]:
# Analyze Amazon
m = analyze_location("Amazon Rainforest", locations["Amazon Rainforest"])
m

## Batch Processing Considerations

When processing multiple regions or large areas, consider:

### Earth Engine Best Practices

1. **Use server-side operations**: Let Earth Engine do the computation
2. **Batch exports**: Use `ee.batch.Export` for large downloads
3. **Tile your analysis**: Break large regions into manageable tiles
4. **Monitor quotas**: Be aware of Earth Engine usage limits

### Example: Exporting Results

In [ ]:
# Example export (uncomment to run)
# Note: Requires proper Earth Engine project setup

# region = ee.Geometry.Rectangle([-122.5, 37.5, -122.0, 38.0])
# 
# task = ee.batch.Export.image.toDrive(
#     image=similarity,
#     description='change_detection_sf',
#     folder='AlphaEarth_Exports',
#     region=region,
#     scale=10,
#     maxPixels=1e9
# )
# 
# task.start()
# print(f"Export started: {task.status()}")

## Integration with Other GIS Workflows

AlphaEarth embeddings can be integrated into broader GIS workflows:

### Combining with Other Datasets

- Overlay with land use/land cover data
- Combine with population density maps
- Integrate with climate data

### Export Formats

- GeoTIFF for desktop GIS (QGIS, ArcGIS)
- Cloud Optimized GeoTIFF (COG) for web applications
- Vector exports for specific regions of interest

In [ ]:
# Example: Combining with land cover
# (Demonstrates concept - adjust dataset IDs as needed)

# # Load land cover data
# land_cover = ee.ImageCollection('ESA/WorldCover/v200').first()
# 
# # Create composite visualization
# m = leafmap.Map(center=[37.8, -122.4], zoom=10)
# m.add_basemap("USGS.Imagery")
# 
# # Add AlphaEarth
# m.add_ee_layer(image_2024, vis_params, "AlphaEarth 2024")
# 
# # Add land cover with transparency
# m.add_ee_layer(land_cover, {'bands': ['Map']}, "Land Cover")
# 
# m.add_layer_control()
# m

## Machine Learning Applications

AlphaEarth embeddings are designed for machine learning tasks:

- **Land Use Classification**: Use embeddings as features for classifiers
- **Semantic Segmentation**: Cluster embeddings to identify land types
- **Anomaly Detection**: Find unusual changes over time
- **Similarity Search**: Find similar locations globally

For detailed ML tutorials, see Google's official documentation:

- [Satellite Embedding Introduction](https://developers.google.com/earth-engine/tutorials/community/satellite-embedding-01-introduction)
- [ArXiv Paper](https://arxiv.org/abs/2507.22291)

## Summary

In this notebook, we covered:

1. **Custom band combinations**: Experiment with different bands to highlight features
2. **Multiple location analysis**: Process several locations systematically
3. **Batch processing**: Considerations for large-scale analysis
4. **Integration**: Combining AlphaEarth with other GIS data and workflows

## Resources

- [AlphaEarth Announcement](https://deepmind.google/blog/alphaearth-foundations-helps-map-our-planet-in-unprecedented-detail/)
- [Earth Engine Dataset Catalog](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_SATELLITE_EMBEDDING_V1_ANNUAL)
- [LeafMap Documentation](https://leafmap.org/maplibre/AlphaEarth/)
- [Research Paper](https://arxiv.org/abs/2507.22291)